In [1]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}   
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, GlobalAveragePooling1D, DepthwiseConv2D, MaxPooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Add
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

IMAGE_SIZE = 96

def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir,'train+val','train')
     valid_path = os.path.join(base_dir,'train+val','valid')


     RESCALING_FACTOR = 1./255

     # instantiate data generators
     datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

os.chdir('..')
dir=os.getcwd()


c:\Users\20222787\.conda\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.5)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


In [2]:
# et the filepath where the trained model is saved (should be in the 'metadata' directory with a .keras extension)
model_name='double_CNN_1'
model_filepath = dir+'/metadata/'+model_name + '_weights.keras'

# load the model from the given filepath
model=tf.keras.models.load_model(model_filepath)

train_gen, val_gen = get_pcam_generators(dir+'\Data')

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [4]:
steps = val_gen.samples // val_gen.batch_size  
loss, accuracy, area_under_curve, recall = model.evaluate(val_gen, steps=steps)

print(f'loss: {loss:.4f}')
print(f'accuracy: {accuracy:.4f}')
print(f'AUC: {area_under_curve:.4f}')
print(f'Recall: {recall:.4f}')

500/500 [==============================] - 148s 280ms/step - loss: 0.3737 - accuracy: 0.8399 - auc_2: 0.9214 - recall_2: 0.8121
loss: 0.3737
accuracy: 0.8399
AUC: 0.9214
Recall: 0.8121


In [8]:
y_pred_prob = list(model.predict(val_gen, batch_size=32).flatten())  # Returns a probability per image
y_pred=[1 if num >= .5 else 0 for num in y_pred_prob]
y_true = list(val_gen.classes)  # The real labels (0 or 1) from the validation set

accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='binary') 

print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')

500/500 [==============================] - 84s 168ms/step
Accuracy: 0.4920
Recall: 0.4642


In [7]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    
# Compute the AUC
roc_auc = auc(fpr, tpr)
print(f'area under curve: {roc_auc:.4f}')

from sklearn.metrics import confusion_matrix as cm
confusion_matrix=cm(y_pred,y_true)
print(confusion_matrix)


area under curve: 0.4987
[[4219 4225]
 [3781 3775]]


In [6]:
y_pred_prob

[0.09647009,
 0.44753748,
 0.45815885,
 0.66050124,
 0.8200889,
 0.009740374,
 0.3608518,
 0.1024749,
 0.3622356,
 0.86885315,
 0.017498277,
 0.38293445,
 0.8587565,
 0.6278922,
 0.23370637,
 0.94011813,
 0.16439728,
 0.6311983,
 0.17019129,
 0.80082196,
 0.31011328,
 0.54424655,
 0.8280437,
 0.32075417,
 0.47446987,
 0.07354232,
 0.8213591,
 0.72888327,
 0.05970204,
 0.14263201,
 0.37908804,
 0.48115826,
 0.1525199,
 0.37478745,
 0.97131145,
 0.25085598,
 0.13542013,
 0.9018046,
 0.8193745,
 0.94007844,
 0.35983983,
 0.29324585,
 0.7211379,
 0.29355463,
 0.33100966,
 0.7124907,
 0.4694538,
 0.41815108,
 0.79978395,
 0.9425037,
 0.10217328,
 0.17306596,
 0.81249404,
 0.214658,
 0.6232567,
 0.66197896,
 0.21109053,
 0.02737412,
 0.19061618,
 0.9539512,
 0.7439414,
 0.7124566,
 0.51088005,
 0.611665,
 0.28872678,
 0.42322075,
 0.450108,
 0.97705525,
 0.09306042,
 0.45301569,
 0.19419183,
 0.8111075,
 0.028191093,
 0.84601074,
 0.1977853,
 0.8363105,
 0.42549944,
 0.58855855,
 0.9662992,
